# BIKE

## Imports


In [36]:
import random as rn
import numpy as np

## Class BIKE

In [65]:
class BIKE:
    def __init__(self, r, t):
        self.K = GF(2)
        self.um = self.K(1)
        self.zero = self.K(0)
        
        self.r = r
        self.n = 2*r
        self.t = t
        
        self.Vn  = VectorSpace(self.K,self.n)
        self.Vr  = VectorSpace(self.K,self.r)
        self.Vq  = VectorSpace(QQ,self.r)
        
        self.Mr  = MatrixSpace(self.K,self.n,self.r)
        
        self.R = PolynomialRing(self.K,name='w')
        self.w = self.R.gen()
        self.Rr = QuotientRing(self.R,self.R.ideal(self.w^self.r+1))
        
    
    # mask ???
    
    # hamm ???
    
    
    def key_gen(self):
        h0 = self.sparse_pol()
        h1 = self.sparse_pol()
        while(not (h0 != h1 and h0.is_unit() and h1.is_unit())):
            h0 = self.sparse_pol()
            h1 = self.sparse_pol()
        private = (h0, h1)
        public = (1, h0/h1)
        return(public, private)    
    
    def enc(self, private, mess):
        (g0, g1) = private
        (e0, e1) = self.noise(self.t)        
        return (mess * g0 + e0, mess * g1 + e1)
    
    def dec(self, public, cripto):
        vec = self.expand2(cripto)
    
        (h0, h1) = public
        H = block_matrix(2,1,[self.Rot(h0),self.Rot(h1)])
        synd = vec * H 
    
        cw = self.BF(H,vec,synd) 
    
        (cw0,cw1) = self.unexpand2(cw)  
    
        return cw0

    
    def rot(self, h):
        v = self.Vr() ; v[0] = h[-1]
        for i in range(self.r-1):
            v[i+1] = h[i]
        return v

    def Rot(self, h):
        M = Matrix(self.K,self.r,self.r) ; M[0] = self.expand(h)
        for i in range(1,self.r):
            M[i] = self.rot(M[i-1])
        return M
    
    def expand(self, f):
        fl = f.list(); ex = self.r - len(fl)
        return self.Vr(fl + [self.zero]*ex)

    def expand2(self, code):
        (f0,f1) = code
        f = self.expand(f0).list() + self.expand(f1).list()
        return self.Vn(f)

    def unexpand2(self, vec):
        u = vec.list()
        return (self.Rr(u[:self.r]),self.Rr(u[self.r:]))
    
    
    def BF(self, H,code,synd, errs=0):
        cnt_iter=self.r
        mycode = code
        mysynd = synd
    
        while cnt_iter > 0 and self.hamm(mysynd) > errs:
            cnt_iter   = cnt_iter - 1
            unsats     = [self.hamm(self.mask(mysynd,H[i])) for i in range(self.n)]
            max_unsats = max(unsats)
        
            for i in range(self.n):
                if unsats[i] == max_unsats:
                    mycode[i]  += self.um               
                    mysynd     += H[i]

        if cnt_iter == 0:
            raise ValueError("BF: limite de iterações ultrapassado")
    
        return mycode   

    
    def sparse_pol(self, sparse=3):
        coeffs = [1]*sparse + [0]*(self.r-2-sparse)
        rn.shuffle(coeffs)
        return self.Rr([1]+coeffs+[1])


    def noise(self, t):
        el = [self.um]*t + [self.zero]*(self.n-self.t)
        rn.shuffle(el)  
        return (self.Rr(el[:self.r]),self.Rr(el[self.r:]))
        
        
    def mask(self, u,v):                                    
        return u.pairwise_product(v)

    def hamm(self, u):                                     
        return sum([1 if a == self.um else 0 for a in u])
    

## Testing

In [67]:
b = BIKE(257, 16)

(g, h) = b.key_gen()
m = b.Rr.random_element()
cr = b.enc(g, m)
d = b.dec(h, cr)
print(cr == m)
print(d == m)

False
True
